In [141]:
import numpy as np
import plotly.graph_objs as go
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from datetime import datetime
import plotly.express as px
from flask import Flask, jsonify, render_template
from config import DB_KEY
import pandas as pd
import plotly
import json

In [2]:
connection_string = "postgres:postgres@localhost:5432/Basketball_stats"
engine = create_engine(f'postgresql://{connection_string}')

games_table_df = pd.read_sql_query('select * from "games"', con=engine)


In [3]:
games_table_df.head()

,game_id,home_team_score,season,visitor_team_score,home_team_id,home_team_full_name,visitor_team_id,visitor_team_full_name,player_id
0,0,47179,126,2018,94,2,Boston Celtics,4,Charlotte Hornets
1,1,48751,112,2018,123,2,Boston Celtics,13,LA Clippers
2,2,48739,117,2018,110,23,Philadelphia 76ers,8,Denver Nuggets
3,3,48740,119,2018,106,30,Washington Wizards,6,Cleveland Cavaliers
4,4,48746,102,2018,96,26,Sacramento Kings,16,Miami Heat


In [4]:
games_data = games_table_df.rename(columns ={'game_id': 'Index', 'home_team_score':'Game ID', 'season':'Home Team Score', 'visitor_team_score': 'Season',
       'home_team_id':'Visitor Team Score', 'home_team_full_name':'Home Team ID', 'visitor_team_id':'Home Team Fullname',
       'visitor_team_full_name':'Visitor Team ID', 'player_id':'Visitor Team Fullname'})
games_data

,Index,Game ID,Home Team Score,Season,Visitor Team Score,Home Team ID,Home Team Fullname,Visitor Team ID,Visitor Team Fullname
0,0,47179,126,2018,94,2,Boston Celtics,4,Charlotte Hornets
1,1,48751,112,2018,123,2,Boston Celtics,13,LA Clippers
2,2,48739,117,2018,110,23,Philadelphia 76ers,8,Denver Nuggets
3,3,48740,119,2018,106,30,Washington Wizards,6,Cleveland Cavaliers
4,4,48746,102,2018,96,26,Sacramento Kings,16,Miami Heat
...,...,...,...,...,...,...,...,...,...
51094,51195,127590,120,2020,122,9,Detroit Pistons,2,Boston Celtics
51095,51196,127633,118,2020,131,17,Milwaukee Bucks,29,Utah Jazz
51096,51197,127569,122,2020,119,11,Houston Rockets,26,Sacramento Kings
51097,51198,127705,120,2020,105,30,Washington Wizards,6,Cleveland Cavaliers


In [5]:
games_data['Season'].min()

1979

In [6]:
games_data = games_data.astype({'Home Team Score':int,'Visitor Team Score':int})

In [7]:
games_data['Winner'] = np.where(games_data ['Home Team Score'] > games_data['Visitor Team Score'],games_data['Home Team Fullname'],games_data['Visitor Team Fullname'])


In [8]:
games_data['Loser'] = np.where(games_data ['Home Team Score'] < games_data['Visitor Team Score'],games_data['Home Team Fullname'],games_data['Visitor Team Fullname'])

In [9]:
games_data.columns

Index(['Index', 'Game ID', 'Home Team Score', 'Season', 'Visitor Team Score',
       'Home Team ID', 'Home Team Fullname', 'Visitor Team ID',
       'Visitor Team Fullname', 'Winner', 'Loser'],
      dtype='object')

In [10]:
winner_count = games_data.groupby(['Season', 'Winner'])['Game ID'].count()

In [11]:
type(winner_count)

pandas.core.series.Series

In [12]:
loser_count = games_data.groupby(['Season', 'Loser'])['Game ID'].count()

In [13]:
loser_count

Season  Loser              
1979    Atlanta Hawks          36
        Boston Celtics         25
        Brooklyn Nets          48
        Chicago Bulls          54
        Cleveland Cavaliers    45
                               ..
2021    Sacramento Kings       50
        San Antonio Spurs      47
        Toronto Raptors        44
        Utah Jazz              36
        Washington Wizards     41
Name: Game ID, Length: 1194, dtype: int64

In [14]:
winner_count

Season  Winner             
1979    Atlanta Hawks          51
        Boston Celtics         66
        Brooklyn Nets          34
        Chicago Bulls          28
        Cleveland Cavaliers    37
                               ..
2021    Sacramento Kings       44
        San Antonio Spurs      34
        Toronto Raptors        49
        Utah Jazz              51
        Washington Wizards     42
Name: Game ID, Length: 1194, dtype: int64

In [15]:
Number_of_Wins= winner_count.to_frame('Number of Wins')

In [16]:
Number_of_Losses = loser_count.to_frame('Number of Losses')

In [17]:
result = pd.concat([Number_of_Wins, Number_of_Losses], axis=1, join="inner")

In [18]:
result

Number of Wins  Number of Losses
Season                                                      
1979   Atlanta Hawks                    51                36
       Boston Celtics                   66                25
       Brooklyn Nets                    34                48
       Chicago Bulls                    28                54
       Cleveland Cavaliers              37                45
...                                    ...               ...
2021   Sacramento Kings                 44                50
       San Antonio Spurs                34                47
       Toronto Raptors                  49                44
       Utah Jazz                        51                36
       Washington Wizards               42                41

[1194 rows x 2 columns]

In [19]:
records_df = result.reset_index()

In [20]:
records_df = records_df.rename(columns= {'level_1':'Teams'})

In [21]:
records_df['Win%'] = round(records_df['Number of Wins']/(records_df['Number of Losses']+records_df['Number of Wins'])*100,2)


In [22]:
season_df = records_df

In [23]:
season_df

,Season,Teams,Number of Wins,Number of Losses,Win%
0,1979,Atlanta Hawks,51,36,58.62
1,1979,Boston Celtics,66,25,72.53
2,1979,Brooklyn Nets,34,48,41.46
3,1979,Chicago Bulls,28,54,34.15
4,1979,Cleveland Cavaliers,37,45,45.12
...,...,...,...,...,...
1189,2021,Sacramento Kings,44,50,46.81
1190,2021,San Antonio Spurs,34,47,41.98
1191,2021,Toronto Raptors,49,44,52.69
1192,2021,Utah Jazz,51,36,58.62


In [24]:
games_df = pd.json_normalize(games_api, record_path =['data'])
games_df

NameError: name 'games_api' is not defined

In [ ]:
season_df.to_sql(name='seasons', con=engine, if_exists='replace', index=True)

In [45]:
season_df.columns

Index(['Season', 'Teams', 'Number of Wins', 'Number of Losses', 'Win%'], dtype='object')

In [168]:
 fig1 = px.bar(season_df, x=['Season'], y='Win%', title='Teams Legacy')

In [156]:
fig1

In [27]:
graph1JSON = json.dumps(fig1, cls = plotly.utils.PlotlyJSONEncoder)

In [171]:
labels = season_df["Teams"].unique()

In [172]:
buttonsLabels = [dict(label = "All labels",
                            method = "restyle",
                            args = [{'y' : labels}] # or what else ?
                            )]

In [173]:
buttonsLabels

[{'label': 'All labels',
  'method': 'restyle',
  'args': [{'y': array(['Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets',
           'Chicago Bulls', 'Cleveland Cavaliers', 'Denver Nuggets',
           'Detroit Pistons', 'Golden State Warriors', 'Houston Rockets',
           'Indiana Pacers', 'LA Clippers', 'Los Angeles Lakers',
           'Milwaukee Bucks', 'New York Knicks', 'Oklahoma City Thunder',
           'Philadelphia 76ers', 'Phoenix Suns', 'Portland Trail Blazers',
           'Sacramento Kings', 'San Antonio Spurs', 'Utah Jazz',
           'Washington Wizards', 'Dallas Mavericks', 'Charlotte Hornets',
           'Miami Heat', 'Minnesota Timberwolves', 'Orlando Magic',
           'Memphis Grizzlies', 'Toronto Raptors', 'New Orleans Pelicans'],
          dtype=object)}]}]

In [174]:
for label in labels:
    buttonsLabels.append(dict(label = label,
                              method = "restyle",
                              visible = True,
                              #args = [{'y' : ??? }]
                              ))

In [175]:
fig1.update_layout(updatemenus = [
   dict(buttons = buttonsLabels, showactive = True)])

In [ ]:
buttonsLabels

In [133]:
season_df(Teams)

NameError: name 'Teams' is not defined

In [178]:
def season_graph(Teams):

    fig1 = go.Figure()

    # drivers = generate_driver_list(year)
    for Team in Teams:
        filter_df = season_df(Team)

        fig1.add_trace(
            go.Scatter(
                x=list(filter_df["Season"]),
                y=list(filter_df["Win%"]),
                Teams = filter_df['Teams'].unqiue(),
                mode="markers"
            )
        )

    fig1.update_layout(
        autosize=False,
        height=600,
        xaxis_title="Seasons",
        yaxis_title="Win %",
        legend_orientation="v",
        legend=dict(x=-0.5, y=0.95),
        xaxis_tickangle=45,
        margin=dict(l=80, r=80, b=120, t=40),
    ) 

In [179]:
fig1